In [1]:
import pyphen
import statistics
import os
import numpy as np
from sklearn.model_selection import train_test_split

from collections import OrderedDict

import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

import nltk

from operator import add

from ordered_set import OrderedSet

import collections

In [11]:
from collections import defaultdict

In [2]:
pos_keys = ['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'INTJ', 'NOUN', 'NUM', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X']


In [63]:
file = "C:\\input\\parsed\\a\\0_a.txt.parsed"
def get_pos_pareameters_list(file, POS):
    pos_properties_list = []
    with open(file, "r", encoding = 'utf-8') as data:
        try:
            for line in data:
                row = line.split('\t')
                if(str(row[0]).isdigit()):
                    if(row[3] == POS):
                        for prop in row[5].split('|'):
                            prop_list = prop.split("=")
                            pos_properties_list.append(prop_list[0])
        except:
            print("get_pos_pareameters_list error raised")
                    
    return pos_properties_list
get_pos_pareameters_list(file,"ADJ")

['Case',
 'Degree',
 'Number',
 'Case',
 'Degree',
 'Derivation',
 'Number',
 '_',
 'Case',
 'Derivation',
 'Number',
 'NumType']

In [4]:
pos_properties_list = ['Abbr', 'Case', 'Clitic', 'Connegative', 'Degree', 'Derivation', 'InfForm', 'Mood', 'Number', 'Number[psor]', 'PartForm', 'Person', 'Person[psor]', 'Polarity', 'Style', 'Tense', 'Typo', 'VerbForm', 'Voice']

In [95]:
file = "C:\\input\\parsed\\a\\0_a.txt.parsed"
def get_pos_properties_values (file, POS, POS_properties_list):
    #print(POS_properties_list)
    properties_dict = dict.fromkeys(POS_properties_list,0)
    properties_ordered_dict = OrderedDict(properties_dict)
    properties_ordered_dict_list = []
    with open(file, "r", encoding = 'utf-8') as data:
        try:
            for line in data:
                row = line.split('\t')
                if(str(row[0]).isdigit()):
                    if(row[3] == POS):
                        for prop in row[5].split('|'):
                            #print(row[1],row[5])
                            prop_list = prop.split("=")
                            try:
                                if(prop_list[0] not in POS_properties_list):
                                    print("key not found, will cretae a new object")
                                properties_ordered_dict[prop_list[0]] = prop_list[1]
                            except:
                                pass
                        properties_ordered_dict_list.append(list(properties_ordered_dict.values()))
            for x in properties_ordered_dict_list:
                #print(len(x), len(POS_properties_list))
                if (len(x) != len(POS_properties_list)):
                    print("wrong length", x)
            if not (properties_ordered_dict_list):
                #print("empty list")
                properties_ordered_dict_list = []
                properties_ordered_dict_list.append(list(properties_ordered_dict.values()))
                return properties_ordered_dict_list
            else:
                #print([len(x) == len(POS_properties_list) for x in properties_ordered_dict_list])
                assert [len(x) == len(POS_properties_list) for x in properties_ordered_dict_list]
        except:
            print("get_pos_properties_values raised error")
    return properties_ordered_dict_list
get_pos_properties_values(file, "VERB", pos_properties_list)

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  'Ind',
  'Plur',
  0,
  0,
  '1',
  0,
  0,
  0,
  'Past',
  0,
  'Fin',
  'Act'],
 [0,
  0,
  0,
  0,
  0,
  0,
  '1',
  'Ind',
  'Sing',
  0,
  0,
  '1',
  0,
  0,
  0,
  'Past',
  0,
  'Inf',
  'Act'],
 [0,
  0,
  0,
  0,
  0,
  0,
  '1',
  'Ind',
  'Sing',
  0,
  0,
  '3',
  0,
  0,
  0,
  'Past',
  0,
  'Fin',
  'Act'],
 [0,
  0,
  0,
  0,
  0,
  0,
  '1',
  'Ind',
  'Sing',
  0,
  0,
  '3',
  0,
  0,
  0,
  'Pres',
  0,
  'Fin',
  'Act'],
 [0,
  'Nom',
  0,
  0,
  'Pos',
  0,
  '1',
  'Ind',
  'Plur',
  0,
  'Past',
  '3',
  0,
  0,
  0,
  'Pres',
  0,
  'Part',
  'Act'],
 [0,
  'Ill',
  0,
  0,
  'Pos',
  0,
  '3',
  'Ind',
  'Sing',
  0,
  'Past',
  '3',
  0,
  0,
  0,
  'Pres',
  0,
  'Inf',
  'Act'],
 [0,
  'Ill',
  0,
  0,
  'Pos',
  0,
  '3',
  'Ind',
  'Plur',
  0,
  'Past',
  '1',
  0,
  0,
  0,
  'Past',
  0,
  'Fin',
  'Act'],
 [0,
  'Nom',
  0,
  0,
  'Pos',
  0,
  '3',
  'Ind',
  'Sing',
  0,
  'Past',
  '1',
  0,
  0,
  0,
  'Past

In [91]:
root = "C:\\input\\parsed"
subfolders_list =['a','b','c']
def get_pos_ohe_properties_list(root, subfolders_list, POS):

    output_data_prep = []
    for folder in subfolders_list:
        for file in os.listdir(os.path.join(root, folder)):
            #print(os.path.join(root, folder, file ))
            if ("parsed" in file):
                unit_data = get_pos_pareameters_list(os.path.join(root, folder, file ),POS)
                output_data_prep.append(unit_data)
                
    pd_data_prep = pd.Series(output_data_prep)
    mlb = MultiLabelBinarizer()
    res = pd.DataFrame(mlb.fit_transform(pd_data_prep),
                       columns=mlb.classes_,
                       index=pd_data_prep.index)
    pos_properties = list(res.columns.values)
    
    #print(res.head(30))
    
    all_pos_properties = []
    for folder in subfolders_list:
        for file in os.listdir(os.path.join(root, folder)):
            #print(os.path.join(root, folder, file ))
            if ("parsed" in file):
                unit_data = get_pos_properties_values(os.path.join(root, folder, file ),POS ,pos_properties)
                all_pos_properties.append(unit_data)
                
    all_pos_properties_by_unit = []
    fistr_unit = all_pos_properties[0][0]
    for unit in all_pos_properties:
        for verb in unit:
            all_pos_properties_by_unit.append(verb)
            if(len(fistr_unit) != len(verb)):
                print(fistr_unit, verb)
            assert len(fistr_unit) == len(verb)
            
    print(len(all_pos_properties_by_unit[0]), len(pos_properties))    
    pos_df = pd.DataFrame(all_pos_properties_by_unit, columns=pos_properties)
    drop_column_name_list = []
    print("show unique elements")
    for ind in range(len(pos_properties)):
        column_unique_values = pos_df.iloc[:,ind].unique()
        print(pos_properties[ind], column_unique_values)
        if(len(column_unique_values) == 1):
            drop_column_name_list.append(pos_properties[ind])
            
            
    #for drop_column_name in drop_column_name_list:
    pos_df = pos_df.drop(columns = drop_column_name_list)
    
    pos_df_ohe = pd.get_dummies(pos_df)
    pos_df_ohe_features = list(pos_df_ohe.columns.values)
    pos_df_ohe_features_set = OrderedSet(pos_df_ohe_features)
    
    """
    print("show unique elements")
    for ind in range(len(pos_properties)):
        print(pos_properties[ind], pos_df.iloc[:,ind].unique())"""
          
    return pos_df_ohe_features_set.items
root = "C:\\input\\parsed"
subfolders_list =['a','b','c']
#get_pos_ohe_properties_list (root, subfolders_list, "CCONJ")

In [1]:
pos_keys = ['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'INTJ', 'NOUN', 'NUM', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X']
pos_keys_dict = dict.fromkeys(pos_keys,[])
for pos in pos_keys:
    print("NOW HANDLING ", pos)
    ohe_features = get_pos_ohe_properties_list(root, subfolders_list, pos)
    print(pos, " features are \n", ohe_features)
    pos_keys_dict[pos] = ohe_features
    print("curretn state of ohe disctionary is\n", pos_keys_dict)

NOW HANDLING  ADJ


NameError: name 'get_pos_ohe_properties_list' is not defined

In [97]:
one_hot_encoded_pos_features_for_dict = {'ADJ': ['Abbr_0', 'Abbr_Yes', 'Case_0', 'Case_Abl', 'Case_Ade', 'Case_All', 'Case_Com', 'Case_Ela', 'Case_Ess', 'Case_Gen', 
         'Case_Ill', 'Case_Ine', 'Case_Ins', 'Case_Nom', 'Case_Par', 'Case_Tra', 'Clitic_0', 'Clitic_Kaan', 'Clitic_Kin', 
         'Clitic_Ko', 'Degree_0', 'Degree_Cmp', 'Degree_Pos', 'Degree_Sup', 'Derivation_0', 'Derivation_Inen', 
         'Derivation_Lainen', 'Derivation_Llinen', 'Derivation_Ton', 'NumType_0', 'NumType_Ord', 'Number_0', 
         'Number_Plur', 'Number_Sing', 'Number[psor]_0', 'Number[psor]_Sing', 'Person[psor]_0', 'Person[psor]_1', 
         'Person[psor]_2', 'Person[psor]_3', 'Style_0', 'Style_Coll', 'Typo_0', 'Typo_Yes'], 
 'ADP': ['AdpType_0', 'AdpType_Post', 'AdpType_Prep', 'Clitic_0', 'Clitic_Kaan', 'Clitic_Kin', 'Number[psor]_0', 
         'Number[psor]_Plur', 'Number[psor]_Sing', 'Person[psor]_0', 'Person[psor]_1', 'Person[psor]_2', 'Person[psor]_3', 
         'Style_0', 'Style_Coll', 'Typo_0', 'Typo_Yes'], 
 'ADV': ['Abbr_0', 'Abbr_Yes', 'Clitic_0', 'Clitic_Han', 'Clitic_Han,Ko', 'Clitic_Kaan', 'Clitic_Kin', 'Clitic_Ko', 
         'Clitic_Ko,S', 'Clitic_Pa', 'Clitic_Pa,S', 'Clitic_S', 'Degree_0', 'Degree_Cmp', 'Degree_Sup', 'Derivation_0', 
         'Derivation_Sti', 'Derivation_Ttain', 'Number[psor]_0', 'Number[psor]_Plur', 'Number[psor]_Sing', 'Person[psor]_0', 
         'Person[psor]_1', 'Person[psor]_2', 'Person[psor]_3', 'Style_0', 'Style_Coll', 'Typo_0', 'Typo_Yes'], 
 'AUX': ['Case_0', 'Case_Abe', 'Case_Ade', 'Case_Ela', 'Case_Ess', 'Case_Gen', 'Case_Ill', 'Case_Ine', 'Case_Ins', 'Case_Nom', 
         'Case_Par', 'Case_Tra', 'Clitic_0', 'Clitic_Han', 'Clitic_Han,Ko', 'Clitic_Han,Pa', 'Clitic_Ka', 'Clitic_Kaan', 
         'Clitic_Kin', 'Clitic_Ko', 'Clitic_Ko,S', 'Clitic_Pa', 'Clitic_Pa,S', 'Connegative_0', 'Connegative_Yes', 'Degree_0', 
         'Degree_Pos', 'InfForm_0', 'InfForm_1', 'InfForm_2', 'InfForm_3', 'Mood_0', 'Mood_Cnd', 'Mood_Imp', 'Mood_Ind', 
         'Mood_Pot', 'Number_0', 'Number_Plur', 'Number_Sing', 'Number[psor]_0', 'Number[psor]_Plur', 'Number[psor]_Sing', 
         'PartForm_0', 'PartForm_Agt', 'PartForm_Past', 'PartForm_Pres', 'Person_0', 'Person_1', 'Person_2', 'Person_3', 
         'Person[psor]_0', 'Person[psor]_1', 'Person[psor]_2', 'Person[psor]_3', 'Polarity_0', 'Polarity_Neg', 'Style_0', 
         'Style_Coll', 'Tense_0', 'Tense_Past', 'Tense_Pres', 'Typo_0', 'Typo_Yes', 'VerbForm_0', 'VerbForm_Fin', 
         'VerbForm_Inf', 'VerbForm_Part', 'Voice_0', 'Voice_Act', 'Voice_Pass'], 'CCONJ': ['Clitic_0', 'Clitic_Ko', 
                                                                                           'Style_0', 'Style_Coll'],
 'INTJ': ['Style_0', 'Style_Coll'], 'NOUN': ['Abbr_0', 'Abbr_Yes', 'Case_0', 'Case_Abe', 'Case_Abl', 'Case_Ade', 'Case_All', 
                                             'Case_Com', 'Case_Ela', 'Case_Ess', 'Case_Gen', 'Case_Ill', 'Case_Ine', 'Case_Ins',
                                             'Case_Nom', 'Case_Par', 'Case_Tra', 'Clitic_0', 'Clitic_Han', 'Clitic_Kaan', 
                                             'Clitic_Kin', 'Degree_0', 'Degree_Pos', 'Derivation_0', 'Derivation_Inen,Vs', 
                                             'Derivation_Ja', 'Derivation_Ja,Tar', 'Derivation_Lainen', 'Derivation_Lainen,Vs',
                                             'Derivation_Llinen,Vs', 'Derivation_Minen', 'Derivation_Tar', 'Derivation_Ton', 
                                             'Derivation_Ton,Vs', 'Derivation_U', 'Derivation_Vs', 'Number_0', 'Number_Plur', 'Number_Sing', 'Number[psor]_0', 'Number[psor]_Plur', 'Number[psor]_Sing', 'Person[psor]_0', 'Person[psor]_1', 'Person[psor]_2', 'Person[psor]_3', 'Style_0', 'Style_Arch', 'Style_Coll', 'Typo_0', 'Typo_Yes'], 'NUM': ['Case_0', 'Case_Abl', 'Case_Ade', 'Case_All', 'Case_Ela', 'Case_Ess', 'Case_Gen', 'Case_Ill', 'Case_Ine', 'Case_Ins', 'Case_Nom', 'Case_Par', 'Case_Tra', 'Clitic_0', 'Clitic_Kaan', 'Clitic_Kin', 'NumType_0', 'NumType_Card', 'Number_0', 'Number_Plur', 'Number_Sing', 'Style_0', 'Style_Coll'], 'PRON': ['Case_0', 'Case_Abl', 'Case_Acc', 'Case_Ade', 'Case_All', 'Case_Com', 'Case_Ela', 'Case_Ess', 'Case_Gen', 'Case_Ill', 'Case_Ine', 'Case_Ins', 'Case_Nom', 'Case_Par', 'Case_Tra', 'Clitic_0', 'Clitic_Han', 'Clitic_Han,Ko', 'Clitic_Kaan', 'Clitic_Kin', 'Clitic_Ko', 'Clitic_Pa', 'Clitic_S', 'Degree_0', 'Degree_Pos', 'Number_0', 'Number_Plur', 'Number_Sing', 'Number[psor]_0', 'Number[psor]_Plur', 'Number[psor]_Sing', 'Person_0', 'Person_1', 'Person_2', 'Person_3', 'Person[psor]_0', 'Person[psor]_1', 'Person[psor]_2', 'Person[psor]_3', 'PronType_0', 'PronType_Dem', 'PronType_Ind', 'PronType_Int', 'PronType_Prs', 'PronType_Rcp', 'PronType_Rel', 'Reflex_0', 'Reflex_Yes', 'Style_0', 'Style_Coll', 'Typo_0', 'Typo_Yes'], 'PROPN': ['Abbr_0', 'Abbr_Yes', 'Case_0', 'Case_Abl', 'Case_Ade', 'Case_All', 'Case_Ela', 'Case_Ess', 'Case_Gen', 'Case_Ill', 'Case_Ine', 'Case_Ins', 'Case_Nom', 'Case_Par', 'Case_Tra', 'Clitic_0', 'Clitic_Kaan', 'Clitic_Kin', 'Clitic_Ko', 'Derivation_0', 'Derivation_Lainen', 'Number_0', 'Number_Plur', 'Number_Sing', 'Number[psor]_0', 'Number[psor]_Sing', 'Person[psor]_0', 'Person[psor]_2', 'Style_0', 'Style_Coll', 'Typo_0', 'Typo_Yes'], 'PUNCT': ['Typo_0', 'Typo_Yes'], 'SCONJ': ['Clitic_0', 'Clitic_Kin', 'Clitic_Ko', 'Style_0', 'Style_Coll'], 'SYM': ['Case_0', 'Case_Par'], 'VERB': ['Abbr_0', 'Abbr_Yes', 'Case_0', 'Case_Abe', 'Case_Abl', 'Case_Ade', 'Case_All', 'Case_Ela', 'Case_Ess', 'Case_Gen', 'Case_Ill', 'Case_Ine', 'Case_Ins', 'Case_Nom', 'Case_Par', 'Case_Tra', 'Clitic_0', 'Clitic_Han', 'Clitic_Han,Ko', 'Clitic_Ka', 'Clitic_Kaan', 'Clitic_Kin', 'Clitic_Ko', 'Clitic_Ko,S', 'Clitic_Pa', 'Clitic_Pa,S', 'Clitic_S', 'Connegative_0', 'Connegative_Yes', 'Degree_0', 'Degree_Pos', 'Degree_Sup', 'Derivation_0', 'Derivation_Ton', 'InfForm_0', 'InfForm_1', 'InfForm_2', 'InfForm_3', 'Mood_0', 'Mood_Cnd', 'Mood_Imp', 'Mood_Ind', 'Mood_Pot', 'Number_0', 'Number_Plur', 'Number_Sing', 'Number[psor]_0', 'Number[psor]_Plur', 'Number[psor]_Sing', 'PartForm_0', 'PartForm_Agt', 'PartForm_Neg', 'PartForm_Past', 'PartForm_Pres', 'Person_0', 'Person_1', 'Person_2', 'Person_3', 'Person[psor]_0', 'Person[psor]_1', 'Person[psor]_2', 'Person[psor]_3', 'Polarity_0', 'Polarity_Neg', 'Style_0', 'Style_Arch', 'Style_Coll', 'Tense_0', 'Tense_Past', 'Tense_Pres', 'Typo_0', 'Typo_Yes', 'VerbForm_0', 'VerbForm_Fin', 'VerbForm_Inf', 'VerbForm_Part', 'Voice_0', 'Voice_Act', 'Voice_Pass'], 'X': ['Foreign_0', 'Foreign_Yes']}

In [ ]:
 {'ADJ': ['Abbr_0', 'Abbr_Yes', 'Case_0', 'Case_Abl', 'Case_Ade', 'Case_All', 'Case_Com', 'Case_Ela', 'Case_Ess', 'Case_Gen', 'Case_Ill', 'Case_Ine', 'Case_Ins', 'Case_Nom', 'Case_Par', 'Case_Tra', 'Clitic_0', 'Clitic_Kaan', 'Clitic_Kin', 'Clitic_Ko', 'Degree_0', 'Degree_Cmp', 'Degree_Pos', 'Degree_Sup', 'Derivation_0', 'Derivation_Inen', 'Derivation_Lainen', 'Derivation_Llinen', 'Derivation_Ton', 'NumType_0', 'NumType_Ord', 'Number_0', 'Number_Plur', 'Number_Sing', 'Number[psor]_0', 'Number[psor]_Sing', 'Person[psor]_0', 'Person[psor]_1', 'Person[psor]_2', 'Person[psor]_3', 'Style_0', 'Style_Coll', 'Typo_0', 'Typo_Yes'], 'ADP': ['AdpType_Post', 'AdpType_Prep', 'Clitic_0', 'Clitic_Kaan', 'Clitic_Kin', 'Number[psor]_0', 'Number[psor]_Plur', 'Number[psor]_Sing', 'Person[psor]_0', 'Person[psor]_1', 'Person[psor]_2', 'Person[psor]_3', 'Style_0', 'Style_Coll', 'Typo_0', 'Typo_Yes'], 'ADV': ['Abbr_0', 'Abbr_Yes', 'Clitic_0', 'Clitic_Han', 'Clitic_Han,Ko', 'Clitic_Kaan', 'Clitic_Kin', 'Clitic_Ko', 'Clitic_Ko,S', 'Clitic_Pa', 'Clitic_Pa,S', 'Clitic_S', 'Degree_0', 'Degree_Cmp', 'Degree_Sup', 'Derivation_0', 'Derivation_Sti', 'Derivation_Ttain', 'Number[psor]_0', 'Number[psor]_Plur', 'Number[psor]_Sing', 'Person[psor]_0', 'Person[psor]_1', 'Person[psor]_2', 'Person[psor]_3', 'Style_0', 'Style_Coll', 'Typo_0', 'Typo_Yes'], 'AUX': ['Case_0', 'Case_Abe', 'Case_Ade', 'Case_Ela', 'Case_Ess', 'Case_Gen', 'Case_Ill', 'Case_Ine', 'Case_Ins', 'Case_Nom', 'Case_Par', 'Case_Tra', 'Clitic_0', 'Clitic_Han', 'Clitic_Han,Ko', 'Clitic_Han,Pa', 'Clitic_Ka', 'Clitic_Kaan', 'Clitic_Kin', 'Clitic_Ko', 'Clitic_Ko,S', 'Clitic_Pa', 'Clitic_Pa,S', 'Connegative_0', 'Connegative_Yes', 'Degree_0', 'Degree_Pos', 'InfForm_0', 'InfForm_1', 'InfForm_2', 'InfForm_3', 'Mood_0', 'Mood_Cnd', 'Mood_Imp', 'Mood_Ind', 'Mood_Pot', 'Number_0', 'Number_Plur', 'Number_Sing', 'Number[psor]_0', 'Number[psor]_Plur', 'Number[psor]_Sing', 'PartForm_0', 'PartForm_Agt', 'PartForm_Past', 'PartForm_Pres', 'Person_0', 'Person_1', 'Person_2', 'Person_3', 'Person[psor]_0', 'Person[psor]_1', 'Person[psor]_2', 'Person[psor]_3', 'Polarity_0', 'Polarity_Neg', 'Style_0', 'Style_Coll', 'Tense_0', 'Tense_Past', 'Tense_Pres', 'Typo_0', 'Typo_Yes', 'VerbForm_Fin', 'VerbForm_Inf', 'VerbForm_Part', 'Voice_0', 'Voice_Act', 'Voice_Pass'], 'CCONJ': ['Clitic_0', 'Clitic_Ko', 'Style_0', 'Style_Coll'], 'INTJ': ['Style_0', 'Style_Coll'], 'NOUN': ['Abbr_0', 'Abbr_Yes', 'Case_0', 'Case_Abe', 'Case_Abl', 'Case_Ade', 'Case_All', 'Case_Com', 'Case_Ela', 'Case_Ess', 'Case_Gen', 'Case_Ill', 'Case_Ine', 'Case_Ins', 'Case_Nom', 'Case_Par', 'Case_Tra', 'Clitic_0', 'Clitic_Han', 'Clitic_Kaan', 'Clitic_Kin', 'Degree_0', 'Degree_Pos', 'Derivation_0', 'Derivation_Inen,Vs', 'Derivation_Ja', 'Derivation_Ja,Tar', 'Derivation_Lainen', 'Derivation_Lainen,Vs', 'Derivation_Llinen,Vs', 'Derivation_Minen', 'Derivation_Tar', 'Derivation_Ton', 'Derivation_Ton,Vs', 'Derivation_U', 'Derivation_Vs', 'Number_0', 'Number_Plur', 'Number_Sing', 'Number[psor]_0', 'Number[psor]_Plur', 'Number[psor]_Sing', 'Person[psor]_0', 'Person[psor]_1', 'Person[psor]_2', 'Person[psor]_3', 'Style_0', 'Style_Arch', 'Style_Coll', 'Typo_0', 'Typo_Yes'], 'NUM': ['Case_0', 'Case_Abl', 'Case_Ade', 'Case_All', 'Case_Ela', 'Case_Ess', 'Case_Gen', 'Case_Ill', 'Case_Ine', 'Case_Ins', 'Case_Nom', 'Case_Par', 'Case_Tra', 'Clitic_0', 'Clitic_Kaan', 'Clitic_Kin', 'NumType_0', 'NumType_Card', 'Number_0', 'Number_Plur', 'Number_Sing', 'Style_0', 'Style_Coll'], 'PRON': ['Case_Abl', 'Case_Acc', 'Case_Ade', 'Case_All', 'Case_Com', 'Case_Ela', 'Case_Ess', 'Case_Gen', 'Case_Ill', 'Case_Ine', 'Case_Ins', 'Case_Nom', 'Case_Par', 'Case_Tra', 'Clitic_0', 'Clitic_Han', 'Clitic_Han,Ko', 'Clitic_Kaan', 'Clitic_Kin', 'Clitic_Ko', 'Clitic_Pa', 'Clitic_S', 'Degree_0', 'Degree_Pos', 'Number_0', 'Number_Plur', 'Number_Sing', 'Number[psor]_0', 'Number[psor]_Plur', 'Number[psor]_Sing', 'Person_0', 'Person_1', 'Person_2', 'Person_3', 'Person[psor]_0', 'Person[psor]_1', 'Person[psor]_2', 'Person[psor]_3', 'PronType_0', 'PronType_Dem', 'PronType_Ind', 'PronType_Int', 'PronType_Prs', 'PronType_Rcp', 'PronType_Rel', 'Reflex_0', 'Reflex_Yes', 'Style_0', 'Style_Coll', 'Typo_0', 'Typo_Yes'], 'PROPN': ['Abbr_0', 'Abbr_Yes', 'Case_0', 'Case_Abl', 'Case_Ade', 'Case_All', 'Case_Ela', 'Case_Ess', 'Case_Gen', 'Case_Ill', 'Case_Ine', 'Case_Ins', 'Case_Nom', 'Case_Par', 'Case_Tra', 'Clitic_0', 'Clitic_Kaan', 'Clitic_Kin', 'Clitic_Ko', 'Derivation_0', 'Derivation_Lainen', 'Number_0', 'Number_Plur', 'Number_Sing', 'Number[psor]_0', 'Number[psor]_Sing', 'Person[psor]_0', 'Person[psor]_2', 'Style_0', 'Style_Coll', 'Typo_0', 'Typo_Yes'], 'PUNCT': ['Typo_0', 'Typo_Yes'], 'SCONJ': ['Clitic_0', 'Clitic_Kin', 'Clitic_Ko', 'Style_0', 'Style_Coll'], 'SYM': ['Case_0', 'Case_Par'], 'VERB': ['Abbr_0', 'Abbr_Yes', 'Case_0', 'Case_Abe', 'Case_Abl', 'Case_Ade', 'Case_All', 'Case_Ela', 'Case_Ess', 'Case_Gen', 'Case_Ill', 'Case_Ine', 'Case_Ins', 'Case_Nom', 'Case_Par', 'Case_Tra', 'Clitic_0', 'Clitic_Han', 'Clitic_Han,Ko', 'Clitic_Ka', 'Clitic_Kaan', 'Clitic_Kin', 'Clitic_Ko', 'Clitic_Ko,S', 'Clitic_Pa', 'Clitic_Pa,S', 'Clitic_S', 'Connegative_0', 'Connegative_Yes', 'Degree_0', 'Degree_Pos', 'Degree_Sup', 'Derivation_0', 'Derivation_Ton', 'InfForm_0', 'InfForm_1', 'InfForm_2', 'InfForm_3', 'Mood_0', 'Mood_Cnd', 'Mood_Imp', 'Mood_Ind', 'Mood_Pot', 'Number_0', 'Number_Plur', 'Number_Sing', 'Number[psor]_0', 'Number[psor]_Plur', 'Number[psor]_Sing', 'PartForm_0', 'PartForm_Agt', 'PartForm_Neg', 'PartForm_Past', 'PartForm_Pres', 'Person_0', 'Person_1', 'Person_2', 'Person_3', 'Person[psor]_0', 'Person[psor]_1', 'Person[psor]_2', 'Person[psor]_3', 'Polarity_0', 'Polarity_Neg', 'Style_0', 'Style_Arch', 'Style_Coll', 'Tense_0', 'Tense_Past', 'Tense_Pres', 'Typo_0', 'Typo_Yes', 'VerbForm_Fin', 'VerbForm_Inf', 'VerbForm_Part', 'Voice_0', 'Voice_Act', 'Voice_Pass'], 'X': ['Foreign_0', 'Foreign_Yes']}

In [100]:
part_of_speech = ['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'INTJ', 'NOUN', 'NUM', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X']


In [98]:
one_hot_encoded_pos_features_for_dict['ADJ']

['Abbr_0',
 'Abbr_Yes',
 'Case_0',
 'Case_Abl',
 'Case_Ade',
 'Case_All',
 'Case_Com',
 'Case_Ela',
 'Case_Ess',
 'Case_Gen',
 'Case_Ill',
 'Case_Ine',
 'Case_Ins',
 'Case_Nom',
 'Case_Par',
 'Case_Tra',
 'Clitic_0',
 'Clitic_Kaan',
 'Clitic_Kin',
 'Clitic_Ko',
 'Degree_0',
 'Degree_Cmp',
 'Degree_Pos',
 'Degree_Sup',
 'Derivation_0',
 'Derivation_Inen',
 'Derivation_Lainen',
 'Derivation_Llinen',
 'Derivation_Ton',
 'NumType_0',
 'NumType_Ord',
 'Number_0',
 'Number_Plur',
 'Number_Sing',
 'Number[psor]_0',
 'Number[psor]_Sing',
 'Person[psor]_0',
 'Person[psor]_1',
 'Person[psor]_2',
 'Person[psor]_3',
 'Style_0',
 'Style_Coll',
 'Typo_0',
 'Typo_Yes']